In [128]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [129]:
wine = datasets.load_wine()
df=pd.DataFrame(wine.data,columns=wine.feature_names)
df.columns = wine.feature_names
target = pd.DataFrame(wine.target,columns=["target"])
df.insert(len(df.columns),"target",target) 
df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


In [130]:
treino, teste = train_test_split(df, test_size=0.3)
treino.reset_index(inplace=True, drop=True); teste.reset_index(inplace=True, drop=True)  
dadosTreino = len(treino)
dadosTeste = len(teste)
totalDf = len(df)
porcTotalTreino = (dadosTreino*100)/totalDf
porcTotalTeste = (dadosTeste*100)/totalDf
print("Dados para treino: ", dadosTreino)
print("Dados para teste: ", dadosTeste)
print("%.2f%% treino" %(porcTotalTreino))
print("%.2f%% teste" %(porcTotalTeste))

Dados para treino:  124
Dados para teste:  54
69.66% treino
30.34% teste


In [131]:
class No():
    def __init__(self, parent=None):
      self.parent = parent
      self.feature = None
      self.filhoEsquerdo = None
      self.filhoDireito = None
      self.target = None
      self.poda = None
    
    def SemDec(self, indiv):
      if self.feature:
        if indiv[self.feature] <= self.poda:
          return self.filhoEsquerdo.SemDec(indiv)
        else: return self.filhoDireito.SemDec(indiv)
      else:
        return self.target

class ID3():
  def __init__(self):
    self.root = None
    self.features = None

  def fit(self, df):
    features = RamoFeatures(df, df.columns.drop("target"))
  
    self.root = No()
    self.CriaArvore(self.root, df, features)
  
  def CriaArvore(self, no, df, features):

      numbCharg = df.target.unique()
      if len(numbCharg) == 1: no.target = numbCharg[0]; return
      
      if len(features) == 0:
        numbCharg = list(df.target)
        no.target = max(set(numbCharg), key = numbCharg.count)
        return
    
      feature = MelhorCarac(df, features)
      features = features.copy()
      features = RamoFeatures(df, features)
      poda = features[feature]
      features.pop(feature)
      
      l_df = df[df[feature] <= poda]
      r_df = df[df[feature] > poda]
      
      if len(l_df) == 0 or len(r_df) == 0:
        numbCharg = list(df.target)
        no.target = max(set(numbCharg), key = numbCharg.count)
        return
        
      no.feature = feature
      no.poda = poda

      no.filhoEsquerdo = No(parent=no)
      no.filhoDireito = No(parent=no)

      self.CriaArvore(no.filhoEsquerdo, l_df, features)
      self.CriaArvore(no.filhoDireito, r_df, features)

      pred_l = no.filhoEsquerdo.target
      pred_r = no.filhoDireito.target
      if pred_l != None and pred_r != None:
        if pred_l == pred_r:
          no.target = pred_l
          no.feature = None
          no.poda = None
          no.filhoEsquerdo = None
          no.filhoDireito = None
  
  def ProcessValidacao(self, df):
    errors = 0
    df_size = len(df)
    for i in df.index:
      indiv = df.loc[i]
      pred = self.root.SemDec(indiv)
      if pred != indiv["target"]:
        errors+=1
    return errors/df_size
  
  def predicao(self, df):
    df_size = len(df)
    for i in range(df_size):
      indiv = df.loc[i]
      pred = self.root.SemDec(indiv)
      indiv["target"] = pred

def MelhorCarac(df, features):
  f = features.copy()
  for i in f:
    poda = f.get(i)
    f[i] = GanhoInformacao(3, df, i, poda)
  return max(f.items(), key=lambda x:x[1])[0]

def RamoFeatures(df, f):
  c = PegaRamos(df, f)
  features = {}
  for i, key  in enumerate(f): features[key] = c[i]
  return features

def PegaRamos(df, features):
  podas = []
  for f in features:
    poda = []
    b = []
    unique =  df[f].sort_values().unique()
    for i, value in enumerate(unique):
      if value == unique[-1]: break
      poda.append((value + unique[i+1])/2)
      b.append(GanhoInformacao(3, df, f, poda[i]))
    podas.append(poda[b.index(max(b))])
  return podas

def entropia(numAlvos, dataset):
  p = []
  somaTotal = 0
  zeros = 0
  for i in range(numAlvos):
    p_i = len(dataset[dataset.target==i])/len(dataset)
    p.append(p_i)   
    if p_i == 0 : zeros += 1
  
  if zeros == numAlvos - 1: return 0

  for i in p:
    if i != 0: somaTotal -= i*np.log2(i)
  
  return somaTotal

def GanhoInformacao(numAlvos, dataset, feature, poda):
  somaGanho = 0
  camB = dataset[dataset[feature] <= poda]
  for i in range(2):
    tamCamB = len(camB)
    if not tamCamB:
      camB = dataset[dataset[feature] > poda]
      continue
    ev = entropia(numAlvos, camB)
    pv = tamCamB / len(dataset)
    somaGanho += pv*ev
    camB = dataset[dataset[feature] > poda]
  return entropia(numAlvos, dataset) - somaGanho


In [132]:
#Faltou mostrar a arvore
arvoreDecisao = ID3()
arvoreDecisao.fit(treino)
acc = ((arvoreDecisao.ProcessValidacao(teste)*100)-100)*-1
print("Acurácia: %.2f%%" %(porcTotalTreino))

Acurácia: 69.66%
